In [1]:
import os
import pandas as pd
import cx_Oracle
from dotenv import load_dotenv

### Inicializálás

In [2]:
load_dotenv()
username = os.getenv('ORACLE_USERNAME')
password = os.getenv('ORACLE_PASSWORD')
dsn = os.getenv('ORACLE_DSN')
pool = cx_Oracle.SessionPool(user=username, password=password, dsn=dsn, min=2, max=5, increment=1)

input_file = 'TXT Ukrán január.xlsx'
output_file = 'elokeszit_Ukrán_202501.xlsx'

AHT='397451'
KID=f'{AHT}/2024'
SHEET='Munka1'

def get_data(ksh):
    query = f'''
select 
    onk.ksh, 
    SUBSTR(st.torzs_sz||'-'||st.afakod||'-'||st.mk, 0, 8) as torzsszam,    
    cim.irszam, 
    onk.onev,
    cim.koztnev||' '||cim.kozjell as cim,                       
    cim.hsz1 as hsz, 
    TRIM(rk.megnevezes) as regio
from SUPPORTSYS.t_onkorm_tech onk
left join (select * from SUPPORTSYS.t_stat) st on st.ksh=onk.ksh
left join (select * from SUPPORTSYS.t_cim) cim on cim.ksh=onk.ksh
left join SUPPORTSYS.t_regio rk on rk.regiokod=onk.rk
where onk.ksh='{ksh}'
'''
    connection = pool.acquire()
    cursor = connection.cursor()
    cursor.execute(query)
    data = cursor.fetchall()    
    pool.release(connection)
    return data

### EBR betöltő beolvasás

In [3]:
df = pd.read_excel(f'./data/{input_file}', sheet_name=0, usecols=[0, 1, 2], names=['KSH', 'ONK', 'AMOUNT'], skiprows=0) #, nrows=41)
df['KSH'] = df['KSH'].astype(str).str.replace('\.0', '', regex=True).str.zfill(7)
# df['KSH'] = df['KSH'].astype(str).str.zfill(7)
df['AMOUNT'] = df['AMOUNT'].fillna(0).astype(int)
df['KID'] = KID
df['TORZSSZAM'] = ''
df['AHT'] = AHT
df['IRSZAM'] = ''
df['ONEV'] = ''
df['CIM'] = ''
df['HSZ'] = ''
df['REGIO'] = ''
if df.iloc[-1]['KSH'] == '9999999':
  df = df.iloc[:-1]

df

,KSH,ONK,AMOUNT,KID,TORZSSZAM,AHT,IRSZAM,ONEV,CIM,HSZ,REGIO
0,1904561,Ábrahámhegy Község Önkormányzata,310000,397451/2024,,397451,,,,,
1,1406080,Ádánd Község Önkormányzata,465000,397451/2024,,397451,,,,,
2,0303522,Baja Város Önkormányzata,310000,397451/2024,,397451,,,,,
3,1520677,Beregsurány Község Önkormányzata,310000,397451/2024,,397451,,,,,
4,1308891,Biatorbágy Város Önkormányzata,1395000,397451/2024,,397451,,,,,
5,1706497,Bonyhád Város Önkormányzata,1085000,397451/2024,,397451,,,,,
6,0113578,Budapest Fővárosi Önkormányzat,6355000,397451/2024,,397451,,,,,
7,0118069,Budapest III. Kerület Önkormányzata,900000,397451/2024,,397451,,,,,
8,0108208,Budapest XVI. Kerület Önkormányzata,620000,397451/2024,,397451,,,,,
9,1530641,Csenger Város Önkormányzata,620000,397451/2024,,397451,,,,,


### Szükséges adatok az adatbázisból

In [4]:
column_names = ['TORZSSZAM', 'IRSZAM', 'ONEV', 'CIM', 'HSZ', 'REGIO']

for index, row in df.iterrows():
    ksh_value = row['KSH']
    data = get_data(ksh_value)
    for i, column_name in enumerate(column_names):
        df.at[index, column_name] = data[0][i+1]
df.head()

,KSH,ONK,AMOUNT,KID,TORZSSZAM,AHT,IRSZAM,ONEV,CIM,HSZ,REGIO
0,1904561,Ábrahámhegy Község Önkormányzata,310000,397451/2024,15427786,397451,8256,Ábrahámhegy,Badacsonyi Út,13,Közép-Dunántúl
1,1406080,Ádánd Község Önkormányzata,465000,397451/2024,15731340,397451,8653,Ádánd,Kossuth Lajos Utca,50,Dél-Dunántúl
2,0303522,Baja Város Önkormányzata,310000,397451/2024,15724313,397451,6500,Baja,Szentháromság Tér,1,Dél-Alföld
3,1520677,Beregsurány Község Önkormányzata,310000,397451/2024,15405140,397451,4933,Beregsurány,Rákóczi Út,1,Észak-Alföld
4,1308891,Biatorbágy Város Önkormányzata,1395000,397451/2024,15730088,397451,2051,Biatorbágy,Baross Gábor Utca,2/a,Budapest / Pest megye (2021.12.31-ig Közép-Mag...


### Bezárás

In [5]:
pool.close()
df.to_excel(f'./data/{output_file}', index=False)